In [1]:
# Libraries
import base64
import io
from openai import OpenAI
from Credentials import *

### Function to Fetch Image from URL

In [2]:
import requests
from PIL import Image

# def getImageFromUrl(url):
#     try:
#         response = requests.get(url, stream=True)
#         response.raise_for_status() # don't know much
#         image = Image.open(response.raw)
#         return image
#     except requests.exceptions.RequestException as e:
#         print(f"Error in getImageFromUrl...take action \n Error: {e}")

In [10]:
from PIL import Image
import requests
import os

def getImage(source):
    try:
        # Check if the source is a URL
        if source.startswith("http://") or source.startswith("https://"):
            response = requests.get(source, stream=True)
            response.raise_for_status()
            image = Image.open(response.raw)
        else:
            # Assume the source is a file path
            if not os.path.exists(source):
                raise FileNotFoundError(f"The file path {source} does not exist.")
            image = Image.open(source)
            
        return image
    except (requests.exceptions.RequestException, FileNotFoundError, IOError) as e:
        print(f"Error in get_image...take action \n Error: {e}")


### Caption Generation

##### Encoding Image using base64 to transmit in api

In [3]:
def encode_image(image):
  buffered = io.BytesIO()
  image.save(buffered, format="PNG")
  image_bytes = buffered.getvalue()
  
  # Encode the image bytes to base64
  return base64.b64encode(image_bytes).decode('utf-8')

In [4]:
client = OpenAI(api_key = OPEN_AI_API_KEY)

def generateImageCaption(base64_image):
    try:
        response = client.chat.completions.create(
            model=IMAGE2TEXTMODEL,
            messages=[
                {
                "role": "user",
                "content": [
                    {
                        "type": "text", 
                        "text": "Describe the image in atmost 50 tokens. Make sure to complete the description. Give me whole context of image with different elements into it"
                    },
                    {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                    },
                ],
                }
            ],
            max_tokens=50,
        )
        return response.choices[0].message.content
    except:
        return "Error in API Calling...Take Action before too it's get too late"

In [11]:
def getImageCaption(url):
    image = getImage(url)
    base64_image = encode_image(image)
    caption = generateImageCaption(base64_image)
    return caption

In [14]:
# caption = getImageCaption("https://static1.bigstockphoto.com/1/5/2/large1500/251357800.jpg")
caption = getImageCaption("/Users/manoj.kuma/Project/Media-Hackathon/Team-MVPs/Assets/Testing_Images/testing_image2.png")

In [15]:
caption

'The image shows a cricketer in action, wearing a blue uniform with an emblem. He is poised to hit the ball with a bat, focusing intently. The stadium is packed with spectators, creating an energetic atmosphere, while advertisements and team branding'